<a href="https://colab.research.google.com/github/skymined/study_data_analytics/blob/main/docs/quests/CDAs/CDA_categorical_categorical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setting

In [1]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 코렙의 경우 설치 후 runtime  > restart 해야함

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic')

## X(범주형) / Y(범주형) 케이스
- 범주형 변수(X와 Y) 간의 관계에 대한 질문은 이러한 변수들 사이의 의존성 또는 연관성을 탐색하는 것을 목표



### 2. 요일(X)이 가장 인기 있는 과일 카테고리 판매(Y)에 영향을 미칩니까?
- 분석 방법: 카이제곱 검정 또는 기대 셀 빈도가 5 미만인 경우 피셔의 정확 검정을 사용하여 특정 요일에 과일 카테고리의 인기에 통계적으로 유의미한 차이가 있는지 평가합니다.

In [7]:
df_fruit = pd.read_csv("fruitstore_saleshistory.csv")
df_fruit[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1.0,2.723699e+08,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207.0,매장C,NaN,10:04
1,2018-01-02,화,1,1.0,1.506656e+09,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977.0,매장C,NaN,10:05


#### 결측치 해결

In [27]:
df_fruit[['요일', '물품대분류']].isnull().sum()
df_fruit.dropna(subset=['물품대분류'] ,inplace=True)
df_fruit['물품대분류']

0            과실
1            채소
2           축산물
3            반찬
4            간식
          ...  
230797       수산
230798    양념/가루
230799     생활용품
230800       반찬
230801      축산물
Name: 물품대분류, Length: 230802, dtype: object

In [30]:
df_fruit[['요일', '물품대분류']].isnull().sum()

요일       0
물품대분류    0
dtype: int64

#### 분석 절차
- 귀무가설 : 요일(X)과 물품대분류(Y)는 관련이 없다. (p>0.05)
- 대립가설 : 요일(X)과 물품대분류(Y)는 관련이 있다. (p<0.05)

In [28]:
# 우선 두 가지의 pivot table을 만들어봅시다.
df_fruit_category_pivot = pd.pivot_table(df_fruit[['요일', '물품대분류']], index="요일", columns='물품대분류', aggfunc=len)
df_fruit_category_pivot

물품대분류,간식,건강,과실,기타,반찬,생활용품,서류,수산,양념/가루,음료,잡곡,주곡,주류,채소,축산물
요일,,,,,,,,,,,,,,,
금,4733.0,266.0,1087.0,2.0,3704.0,866.0,583.0,2644.0,1387.0,1075.0,398.0,377.0,41.0,6972.0,5678.0
목,5320.0,308.0,1136.0,NaN,3744.0,949.0,564.0,2789.0,1672.0,1284.0,467.0,417.0,43.0,7151.0,5834.0
수,6708.0,358.0,1592.0,NaN,4903.0,1225.0,721.0,3678.0,2247.0,1692.0,657.0,518.0,69.0,10028.0,7435.0
월,6494.0,410.0,1482.0,NaN,5080.0,1362.0,768.0,3331.0,2234.0,1545.0,638.0,500.0,65.0,9005.0,7418.0
일,2890.0,163.0,655.0,2.0,1971.0,689.0,333.0,1273.0,909.0,776.0,271.0,316.0,28.0,3343.0,2917.0
토,4754.0,279.0,1201.0,4.0,3451.0,981.0,600.0,2261.0,1402.0,1107.0,453.0,505.0,52.0,7378.0,5284.0
화,6673.0,522.0,1417.0,3.0,5071.0,1294.0,712.0,3308.0,2096.0,1651.0,719.0,547.0,44.0,9417.0,7426.0


In [33]:
df_fruit_category_pivot['기타'] = df_fruit_category_pivot['기타'].fillna(0)

In [34]:
import scipy.stats as stats
stats.chi2_contingency(observed=df_fruit_category_pivot, correction=True)

Chi2ContingencyResult(statistic=621.4895156480367, pvalue=5.896603580394322e-83, dof=84, expected_freq=array([[4.85322500e+03, 2.97869074e+02, 1.10699825e+03, 1.42088457e+00,
        3.60698006e+03, 9.51475975e+02, 5.52982439e+02, 2.49093982e+03,
        1.54320981e+03, 1.17933419e+03, 4.65404282e+02, 4.10764811e+02,
        4.41765929e+01, 6.88405656e+03, 5.42416225e+03],
       [5.15682627e+03, 3.16502751e+02, 1.17624830e+03, 1.50977028e+00,
        3.83262048e+03, 1.01099708e+03, 5.87575142e+02, 2.64676455e+03,
        1.63974778e+03, 1.25310933e+03, 4.94518392e+02, 4.36460863e+02,
        4.69401305e+01, 7.31469975e+03, 5.76347942e+03],
       [6.80962181e+03, 4.17943891e+02, 1.55324334e+03, 1.99366123e+00,
        5.06099966e+03, 1.33502806e+03, 7.75896704e+02, 3.49506938e+03,
        2.16529734e+03, 1.65473882e+03, 6.53014675e+02, 5.76349338e+02,
        6.19847402e+01, 9.65910743e+03, 7.61071114e+03],
       [6.56560127e+03, 4.02967011e+02, 1.49758338e+03, 1.92221904e+00,
      

#### 분석결과 (기준 p > 0.05)
- 통계 측면 결론 : 귀무가설 성립, 대립가설 거짓
- 사실 결론 : 요일과 가장 인기 있는 과일 카테고리 판매는 연관이 없다.